In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")


C:\Users\User\anaconda3\lib\site-packages\dask\dataframe\backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\User\anaconda3\lib\site-packages\dask\dataframe\backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\User\anaconda3\lib\site-packages\dask\dataframe\backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

...

ImportError: numpy.core.umath failed to import

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_combine_data = pd.read_csv('train_combine.csv')

train_data.head(10)

In [ ]:
train_combine_data.info()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
# Reviewing Skew of Attribute Distribution
skew_value = train_data.skew()

skewness_values = pd.DataFrame({
    'Variable': train_data.columns,
    'Skewness': skew_value
})
plt.figure(figsize=(50,8))
# Plot the skewness values using a barplot
sns.barplot(x='Variable', y='Skewness', data=skewness_values)

# Add labels and a title to the plot

plt.xlabel("Features",size=15)
plt.ylabel("Skewness",size=15)
plt.title("Skewness of Variables in bDiabetes Dataset",size=20)

plt.show()

In [ ]:
# Histrogram for each attribute in dataset
train_data.hist(figsize=(20,20),column=train_data.drop(columns=['id']).columns,color='purple')
plt.show()

In [ ]:
X = train_combine_data.drop(columns=['id','yield'])
y = train_combine_data[['yield']]
X_test = test_data.drop(columns=['id']) 

### Feature Selection Method

In [ ]:
# Reviewing Correlation between attributes
correlations = round(X.corr(),4)
correlations # 1 represent full positive correlation, -1 represent negative correllation

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(correlations, annot=True,cmap ='RdBu_r')
plt.show()

In [ ]:
# Loop over bottom diagonal of correlation matrix
threshold = 0.9
high_corr = []
for i in range(len(correlations.columns)):
    for j in range(i):
        if abs(correlations.iloc[i, j]) > threshold:
            colname = correlations.columns[i]
            high_corr.append(colname)
            
set(high_corr)

# remove highly correlated features
# df.drop(high_corr, axis=1, inplace=True)

In [ ]:
# Merge features and targets together for checking correlation
X_y = X.copy()
X_y['yield'] = y
X_y

In [ ]:
correlation = X_y.corr()
correlation_target = correlation[['yield']].drop(labels=['yield'])
correlation_target

In [ ]:
X["fruit_seed"] = X["fruitset"] * X["seeds"]
X["pollinators"] = X["honeybee"] + X["bumbles"] + X["andrena"] + X["osmia"]
X = X.drop(columns=['RainingDays', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'honeybee', 'bumbles', 'andrena' , 'osmia'])

X_test["fruit_seed"] = X_test["fruitset"] * X_test["seeds"]
X_test["pollinators"] = X_test["honeybee"] + X_test["bumbles"] + X_test["andrena"] + X_test["osmia"]
X_test = X_test.drop(columns=['RainingDays', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'honeybee', 'bumbles', 'andrena' , 'osmia'])

print(X.columns)
print(X_test.columns)

In [ ]:
# X_original = X.drop(columns=['AverageOfLowerTRange','AverageOfUpperTRange','MaxOfLowerTRange','MinOfLowerTRange','MinOfUpperTRange'])
# X_test_original = X_test.drop(columns=['AverageOfLowerTRange','AverageOfUpperTRange','MaxOfLowerTRange','MinOfLowerTRange','MinOfUpperTRange'])
# print(X_original.columns)
# print(X_test_original.columns)

In [ ]:
sc = StandardScaler()
X_scaler = pd.DataFrame(sc.fit_transform(X),columns=[X.columns])
X_scaler

### Feature Selection

In [ ]:
def feature_selection(model,x_data):
    
    # Ensure the random distributions are same when execute 
    np.random.seed(0)
    
    sfs = SFS(model,
          k_features=8,
          forward=True,
          floating=False,
          scoring='neg_mean_absolute_error',
          cv=3)
    
    sfs.fit(x_data, y)
    
    print('Feature selection: \n\n')
    for idx, score in sfs.subsets_.items():
        print(idx,':', score, '\n')
    
    print('Model MAE: \n\n')
    for idx, score in sfs.subsets_.items():
        print(idx,':', -1 * score['avg_score'].round(4))  # multiply by -1 to convert back to MAE score

    
    plot_sfs(sfs.get_metric_dict())
    plt.grid()
    plt.title('Accuracy vs Number of Features')
    plt.show()

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(X,y,test_size=.2,random_state=36,shuffle=True)

print(features_train.shape)
print(features_test.shape)
print(labels_train.shape)
print(labels_test.shape)

In [ ]:
print(X)

### XGBoost Model

In [ ]:
# xgboost_model = XGBRegressor()

# feature_selection(xgboost_model,X)

In [ ]:
# X_data = X[['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'RainingDays', 'fruitset', 'fruitmass', 'seeds']]
# min_mae_xgboost = []

# for i in range(42):
#     features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=i,shuffle=True)

#     xgboost_model = XGBRegressor(subsample= 0.7, 
#                                  reg_lambda=0.1, 
#                                  reg_alpha=0.1, 
#                                  objective='reg:squarederror', 
#                                  n_estimators=1000, 
#                                  min_child_weight=5, 
#                                  max_depth= 4, 
#                                  learning_rate= 0.01, 
#                                  gamma=0.2, 
#                                  eval_metric='mae', 
#                                  colsample_bytree=0.9)

#     xgboost_model.fit(features_train, labels_train)
#     predicted = xgboost_model.predict(features_test)
#     mae = mean_absolute_error(labels_test, predicted)
#     min_mae_xgboost.append(mae)

# for i,value in enumerate(min_mae_xgboost):
#     if value == min(min_mae_xgboost):
#         print("Mean Absolute Error:", min(min_mae_xgboost), 'random_state:', i)

### LGBM Model

In [ ]:
# lgbm_model = LGBMRegressor()

# feature_selection(lgbm_model,X)

In [ ]:
# X_data = X[['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'RainingDays', 'fruitset', 'fruitmass', 'seeds']]
# features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=36,shuffle=True)


# lgbm_model = LGBMRegressor(subsample= 0.9, 
#                            reg_lambda=0, 
#                            reg_alpha= 0.3, 
#                            objective = 'mae', 
#                            num_leaves = 64, 
#                            n_estimators= 1000, 
#                            min_child_weight= 15, 
#                            max_depth= 5, 
#                            learning_rate= 0.05,
#                            colsample_bytree= 0.9, 
#                            force_col_wise= True
#                           )

# parameters = {
#               'min_child_samples':[25,20,15,10,5],
#               'subsample_freq':[0,2,4,6,8,10],
#               'max_bin':[128,150,200,255],
#             }

# grid = GridSearchCV(lgbm_model,parameters,scoring='neg_mean_absolute_error',cv=3)
# grid.fit(features_train, labels_train)

# print("Best parameters:", grid.best_params_)
# print("Best score of training data:", round(grid.best_score_*100,2), "%\n")

# # predict on test data with best hyperparameters
# y_pred = grid.predict(features_test)

# # calculate MAE on test data with best hyperparameters
# mae = mean_absolute_error(labels_test, y_pred)
# print("Test MAE with Best Hyperparameters: ", mae)

In [ ]:
# X_data = X[['clonesize', 'AverageOfLowerTRange', 'AverageRainingDays', 'fruitset',
#        'fruitmass', 'seeds', 'fruit_seed', 'pollinators']]
# min_mae_lgbm = []

# for i in range(42):
    
#     features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=i,shuffle=True)
#     lgbm_model = LGBMRegressor(subsample= 0.9, 
#                                reg_lambda=0, 
#                                reg_alpha= 0.3, 
#                                objective = 'mae', 
#                                num_leaves = 64, 
#                                n_estimators= 1000, 
#                                min_child_weight= 15, 
#                                max_depth= 5, 
#                                learning_rate= 0.05,
#                                colsample_bytree= 0.9, 
#                                force_col_wise= True,
#                               )

#     lgbm_model.fit(features_train, labels_train)
#     predicted = lgbm_model.predict(features_test)
#     mae = mean_absolute_error(labels_test, predicted)
#     min_mae_lgbm.append(mae)
    
# for i,value in enumerate(min_mae_lgbm):
#     if value == min(min_mae_lgbm):
#         print("Mean Absolute Error:", min(min_mae_lgbm), 'random_state:', i)

In [ ]:
X_data = X[['clonesize', 'AverageOfLowerTRange', 'AverageRainingDays', 'fruitset',
       'fruitmass', 'seeds', 'fruit_seed', 'pollinators']]

features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=1,shuffle=True)

lgbm_model_2 = LGBMRegressor(subsample= 0.9, 
                               reg_lambda=0, 
                               reg_alpha= 0.3, 
                               objective = 'mae', 
                               num_leaves = 64, 
                               n_estimators= 1000, 
                               min_child_weight= 15, 
                               max_depth= 5, 
                               learning_rate= 0.05,
                               colsample_bytree= 0.9, 
                               force_col_wise= True,
                              )

lgbm_model_2.fit(features_train, labels_train)
lgbm_predicted_2 = lgbm_model_2.predict(features_test)
lgbm_mae_2 = mean_absolute_error(labels_test, lgbm_predicted_2)
print(lgbm_mae_2)

## CatBoost Model

In [ ]:
# cat_model = CatBoostRegressor(verbose=False)

# feature_selection(cat_model,X)

In [ ]:
cat_model = CatBoostRegressor(verbose=False, 
                                  subsample=0.7,
                                  random_strength=0.1, 
                                  loss_function='MAE', 
                                  learning_rate=0.05, 
                                  l2_leaf_reg=1, 
                                  iterations=1000,
                                  depth= 16, 
                                  colsample_bylevel= 0.9, 
                                  grow_policy="Lossguide",
                                  border_count=980,
                                  max_leaves=35
                               )
                               

parameters = {
                 'max_leaves':[x for x in range(20,40,1)]
            }

grid = GridSearchCV(cat_model,parameters,scoring='neg_mean_absolute_error',cv=3)
grid.fit(features_train, labels_train)

print("Best parameters:", grid.best_params_)
print("Best score of training data:", round(grid.best_score_*100,2), "%\n")

# predict on test data with best hyperparameters
y_pred = grid.predict(features_test)

# calculate MAE on test data with best hyperparameters
mae = mean_absolute_error(labels_test, y_pred)
print("Test MAE with Best Hyperparameters: ", mae)

In [ ]:
X_data = X[['clonesize', 'AverageOfLowerTRange', 'AverageRainingDays', 'fruitset',
       'fruitmass', 'seeds', 'fruit_seed', 'pollinators']]

features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=36,shuffle=True)

# cb_params = {
# 'verbose': False, 'subsample': 0.5, 'random_strength': 0.05, 'loss_function': 'MAE', 'learning_rate': 0.05, 'l2_leaf_reg': 0, 'iterations': 1000, 'grow_policy': 'Lossguide', 'depth': 16, 'colsample_bylevel': 0.7, 'bagging_temperature': 0.1
#         }
# cat_model_2 = CatBoostRegressor(**cb_params)
cat_model_2 = CatBoostRegressor(verbose=False, 
                                  subsample=0.7,
                                  random_strength=0.1, 
                                  loss_function='MAE', 
                                  learning_rate=0.05, 
                                  l2_leaf_reg=1, 
                                  iterations=1000,
                                  depth= 16, 
                                  colsample_bylevel= 0.9, 
                                  grow_policy="Lossguide",
                                  border_count=980,
                                  max_leaves=31
                               )


cat_model_2.fit(features_train, labels_train)
cat_predicted_2 = cat_model_2.predict(features_test)
cat_mae_2 = mean_absolute_error(labels_test, cat_predicted_2)
print(cat_mae_2)

In [ ]:
# from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import LinearRegression,Ridge
# from sklearn.ensemble import GradientBoostingRegressor

# X_data = X[['clonesize', 'AverageOfLowerTRange', 'AverageRainingDays', 'fruitset',
#        'fruitmass', 'seeds', 'fruit_seed', 'pollinators']]

# features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=36,shuffle=True)

# cat_model_2 = CatBoostRegressor(verbose=False, 
#                                   subsample=0.7,
#                                   random_strength=0.1, 
#                                   loss_function='MAE', 
#                                   learning_rate=0.05, 
#                                   l2_leaf_reg=1, 
#                                   iterations=1000,
#                                   depth= 16, 
#                                   colsample_bylevel= 0.9, 
#                                   grow_policy="Lossguide",
#                                   border_count=980
#                                )

# lgbm_model_2 = LGBMRegressor(subsample= 0.9, 
#                                reg_lambda=0, 
#                                reg_alpha= 0.3, 
#                                objective = 'mae', 
#                                num_leaves = 64, 
#                                n_estimators= 1000, 
#                                min_child_weight= 15, 
#                                max_depth= 5, 
#                                learning_rate= 0.05,
#                                colsample_bytree= 0.9, 
#                                force_col_wise= True,
#                               )

# meta_model_list = [LinearRegression(), CatBoostRegressor(),Ridge(alpha=1.0),GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)]
# meta_model_1 = LinearRegression(),
# meta_model_2 = CatBoostRegressor(verbose=False)
# meta_model_3 = Ridge(alpha=1.0)
# meta_model_4 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)

# print(meta_model_list)


# for i in meta_model_list:

#     stacked_model = StackingRegressor(
#         estimators=[('cat', cat_model_2), ('lgbm', lgbm_model_2)],
#         final_estimator=i
#     )

#     stacked_model.fit(features_train, labels_train)
#     stacked_predicted = stacked_model.predict(features_test)
#     stacked_mae = mean_absolute_error(labels_test, stacked_predicted)
#     print(stacked_mae)


In [ ]:
# from sklearn.model_selection import KFold
# import numpy as np
# from sklearn.linear_model import LinearRegression

# X_data = X[['clonesize', 'AverageOfLowerTRange', 'AverageRainingDays', 'fruitset',
#        'fruitmass', 'seeds', 'fruit_seed', 'pollinators']]
# features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=36,shuffle=True)

# cat_model_2 = CatBoostRegressor(verbose=False, 
#                                   subsample=0.7,
#                                   random_strength=0.1, 
#                                   loss_function='MAE', 
#                                   learning_rate=0.05, 
#                                   l2_leaf_reg=1, 
#                                   iterations=1000,
#                                   depth= 16, 
#                                   colsample_bylevel= 0.9, 
#                                   grow_policy="Lossguide",
#                                   border_count=980
#                                )

# lgbm_model_2 = LGBMRegressor(subsample= 0.9, 
#                                reg_lambda=0, 
#                                reg_alpha= 0.3, 
#                                objective = 'mae', 
#                                num_leaves = 64, 
#                                n_estimators= 1000, 
#                                min_child_weight= 15, 
#                                max_depth= 5, 
#                                learning_rate= 0.05,
#                                colsample_bytree= 0.9, 
#                                force_col_wise= True,
#                               )


# meta_model_1 = LinearRegression()

# stacked_model = StackingRegressor(
#         estimators=[('cat', cat_model_2), ('lgbm', lgbm_model_2)],
#         final_estimator=meta_model_1
#     )

# stacked_model.fit(features_train, labels_train)
# stacked_predicted = stacked_model.predict(features_test)
# stacked_mae = mean_absolute_error(labels_test, stacked_predicted)
# print(stacked_mae)



In [ ]:
# X_data = X_original[['clonesize', 'AverageOfLowerTRange', 'AverageRainingDays', 'fruitset',
#        'fruitmass', 'seeds', 'fruit_seed', 'pollinators']]

# features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=36)

# cat_model_3 = CatBoostRegressor(verbose=False, 
#                                   subsample=0.7, 
#                                   random_strength=0.1, 
#                                   loss_function='MAE', 
#                                   learning_rate=0.05, 
#                                   l2_leaf_reg=1, 
#                                   iterations=1010, 
#                                   depth= 16, 
#                                   colsample_bylevel= 0.9, 
#                                   grow_policy="Lossguide",
#                                   border_count=1000,
#                                   leaf_estimation_iterations=1,
#                                   one_hot_max_size=1  
#                                )

# cat_model_3.fit(features_train, labels_train)
# cat_predicted_3 = cat_model_3.predict(features_test)
# cat_mae_3 = mean_absolute_error(labels_test, cat_predicted_3)
# print(cat_mae_3)

In [ ]:
# X_data = X[['clonesize', 'AverageOfLowerTRange', 'AverageRainingDays', 'fruitset',
#        'fruitmass', 'seeds', 'fruit_seed', 'pollinators']]

# features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=36,shuffle=True)

# # cb_params = {
# # 'verbose': False, 'subsample': 0.5, 'random_strength': 0.05, 'loss_function': 'MAE', 'learning_rate': 0.05, 'l2_leaf_reg': 0, 'iterations': 1000, 'grow_policy': 'Lossguide', 'depth': 16, 'colsample_bylevel': 0.7, 'bagging_temperature': 0.1
# #         }
# # cat_model_2 = CatBoostRegressor(**cb_params)
# cat_model_2 = CatBoostRegressor(verbose=False, 
#                                   subsample=0.7,
#                                   random_strength=0.1, 
#                                   loss_function='MAE', 
#                                   learning_rate=0.05, 
#                                   l2_leaf_reg=1.001, 
#                                   iterations=980,
#                                   depth= 16, 
#                                   colsample_bylevel= 0.9, 
#                                   grow_policy="Lossguide",
#                                   border_count=980
#                                )


# cat_model_2.fit(features_train, labels_train)
# cat_predicted_2 = cat_model_2.predict(features_test)
# cat_mae_2 = mean_absolute_error(labels_test, cat_predicted_2)
# print(cat_mae_2)

In [ ]:
# X_data = X[['clonesize', 'honeybee', 'bumbles', 'osmia', 'MaxOfUpperTRange', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds']]
# features_train, features_test, labels_train, labels_test = train_test_split(X_data,y,test_size=.2,random_state=6,shuffle=True)


# model = Sequential()
# model.add(InputLayer(input_shape=(X.shape[1],)))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# opt = Adam(learning_rate=0.01)
# model.compile(optimizer=opt, loss='mse', metrics=['mae'])

# model.fit(features_train,labels_train,batch_size=20, epochs=40, verbose=0)
# #evaluate the model on the test data
# val_mse, val_mae = model.evaluate(features_test, labels_test, verbose=0)

# print("MAE: ", val_mae)

In [ ]:
# def do_grid_search():
    
#     batch_size = [20, 40, 60]
#     epochs = [10, 20, 30, 40, 50]
#     model = KerasRegressor(build_fn=design_model())
#     param_grid = dict(batch_size=batch_size, epochs=epochs)
#     grid = GridSearchCV(estimator = model, param_grid=param_grid, scoring = make_scorer(mean_squared_error, greater_is_better=False),return_train_score = True)
#     grid_result = grid.fit(features_train, labels_train, verbose = 0)
#     print(grid_result)
#     print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# #     means = grid_result.cv_results_['mean_test_score']
# #     stds = grid_result.cv_results_['std_test_score']
# #     params = grid_result.cv_results_['params']
# #     for mean, stdev, param in zip(means, stds, params):
# #         print("%f (%f) with: %r" % (mean, stdev, param))
    
# do_grid_search()

# # model.fit(X_train,y_train,batch_size=10, epochs=40, verbose=0)
# # #evaluate the model on the test data
# # val_mse, val_mae = model.evaluate(X_test, y_test, verbose=0)

# # print("MAE: ", val_mae)

In [ ]:
# model.fit(X_train,y_train,batch_size=20, epochs=10, verbose=0)
# #evaluate the model on the test data
# val_mse, val_mae = model.evaluate(X_test, y_test, verbose=0)

# print("MAE: ", val_mae)

### Write Data to CSV

In [ ]:
X_train = X
y_train = y
X_test = X_test
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

In [ ]:
print(X_train.columns)
print(X_test.columns)
final_model = cat_model_2.fit(X_train,y_train)
final_predicted = final_model.predict(X_test)
column1 = test_data['id']
column2 = list(final_predicted)
dataset=pd.DataFrame({'id':column1,'yield':column2})
dataset.to_csv('submission.csv',index=False)